# Despliegue del modelo con FastAPI


Define un servicio FastAPI listo para ejecutarse desde el propio notebook. Recuerda ejecutar la última celda solo cuando necesites arrancar el servidor.


In [4]:
import json
from pathlib import Path

import joblib
import pandas as pd
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field

CONFIG_PATH = Path("config.json")
if not CONFIG_PATH.exists():
    CONFIG_PATH = Path("../../config.json").resolve()

with CONFIG_PATH.open(encoding="utf-8") as cfg_file:
    config = json.load(cfg_file)

project_root = CONFIG_PATH.parent
preprocessor = joblib.load(project_root / config["paths"]["preprocessor"])
best_payload = joblib.load(project_root / config["paths"]["best_model"])
model = best_payload["model"] if isinstance(best_payload, dict) else best_payload
feature_order = config["features"]["categorical"] + config["features"]["numeric"]

class StudentFeatures(BaseModel):
    gender: str = Field(..., description="Genero del estudiante")
    race_ethnicity: str = Field(..., description="Grupo racial/étnico (group A-E)")
    parental_level_of_education: str = Field(..., description="Nivel educativo de los tutores")
    lunch: str = Field(..., description="Tipo de almuerzo")
    test_preparation_course: str = Field(..., description="Estado del curso de preparación")
    reading_score: float = Field(..., ge=0, le=100, description="Puntaje de lectura")
    writing_score: float = Field(..., ge=0, le=100, description="Puntaje de escritura")

app = FastAPI(title="Students Performance API", version="1.0.0")

@app.get("/health")
def healthcheck():
    return {"status": "ok"}

@app.post("/predict")
def predict(features: StudentFeatures):
    try:
        df = pd.DataFrame([features.dict()])
        df = df[feature_order]
        transformed = preprocessor.transform(df)
        prediction = float(model.predict(transformed)[0])
        return {"pred_math_score": round(prediction, 2)}
    except ValueError as err:
        raise HTTPException(status_code=400, detail=str(err))
    except Exception as exc:  # pylint: disable=broad-except
        raise HTTPException(status_code=500, detail=str(exc))


Ejecuta la celda siguiente para levantar el servidor local. Interrumpe el kernel para detenerlo.


In [5]:
import nest_asyncio, uvicorn

nest_asyncio.apply()

config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

await server.serve()


INFO:     Started server process [27724]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62792 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56381 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56381 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56381 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:49584 - "GET /rutas HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:49584 - "GET /redoc HTTP/1.1" 200 OK
INFO:     127.0.0.1:49584 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:49852 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:49852 - "GET /openapi.json HTTP/1.1" 200 OK


C:\Users\juanp\AppData\Local\Temp\ipykernel_27724\3283642814.py:40: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  df = pd.DataFrame([features.dict()])


INFO:     127.0.0.1:65139 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:51430 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56846 - "GET /docs3 HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56846 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56846 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [27724]
